<a href="https://colab.research.google.com/github/DanYoung321/Pokemon-Image-Classification/blob/main/Pokemon_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Collab: Computer Vision Pokedex

For this project...

USE THIS AS REFERENCE!!!
https://medium.com/analytics-vidhya/pok%C3%A9mon-classification-974a10621381

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import cv2

## Import Dataset from Kaggle

Note: This code requires you to have Kaggle setup on your google drive account.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# WARNING: this code is used to import dataset from kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
# Change the Working Directory
%cd gdrive/My Drive/Kaggle
%ls

[Errno 2] No such file or directory: 'gdrive/My Drive/Kaggle'
/content/gdrive/My Drive/Kaggle
images/  kaggle.json  pokemon.csv  Pokemon_Image_Dataset_v2/  pokemon-images-and-types.zip


Import the *Pokemon Image Dataset*.



In [ ]:
!kaggle datasets download -d vishalsubbiah/pokemon-images-and-types

  0% 0.00/3.68M [00:00<?, ?B/s]
100% 3.68M/3.68M [00:00<00:00, 67.9MB/s]


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip

Archive:  pokemon-images-and-types.zip
  inflating: images/abomasnow.png    
  inflating: images/abra.png         
  inflating: images/absol.png        
  inflating: images/accelgor.png     
  inflating: images/aegislash-blade.png  
  inflating: images/aerodactyl.png   
  inflating: images/aggron.png       
  inflating: images/aipom.png        
  inflating: images/alakazam.png     
  inflating: images/alomomola.png    
  inflating: images/altaria.png      
  inflating: images/amaura.png       
  inflating: images/ambipom.png      
  inflating: images/amoonguss.png    
  inflating: images/ampharos.png     
  inflating: images/anorith.png      
  inflating: images/araquanid.png    
  inflating: images/arbok.png        
  inflating: images/arcanine.png     
  inflating: images/arceus.png       
  inflating: images/archen.png       
  inflating: images/archeops.png     
  inflating: images/ariados.png      
  inflating: images/armaldo.png      
  inflating: images/aromatisse.png   
  infla

## Define Classes

... idk put something here

In [ ]:
from posixpath import splitext
#creating test dataset from the train dataset
dir_path='/content/gdrive/MyDrive/Kaggle/images'
classes = os.listdir(dir_path)

classes = ['{0}'.format(element.split(".png")[0]) for element in classes]

print(classes)

['abomasnow', 'abra', 'absol', 'accelgor', 'aegislash-blade', 'aerodactyl', 'aggron', 'aipom', 'alakazam', 'alomomola', 'altaria', 'amaura', 'ambipom', 'amoonguss', 'ampharos', 'anorith', 'araquanid', 'arbok', 'arcanine', 'arceus', 'archen', 'archeops', 'ariados', 'armaldo', 'aromatisse', 'aron', 'articuno', 'audino', 'aurorus', 'avalugg', 'axew', 'azelf', 'azumarill', 'azurill', 'bagon', 'baltoy', 'banette', 'barbaracle', 'barboach', 'basculin-red-striped', 'bastiodon', 'bayleef', 'beartic', 'beautifly', 'beedrill', 'beheeyem', 'beldum', 'bellossom', 'bellsprout', 'bergmite', 'bewear', 'bibarel', 'bidoof', 'binacle', 'bisharp', 'blacephalon', 'blastoise', 'blaziken', 'blissey', 'blitzle', 'boldore', 'bonsly', 'bouffalant', 'bounsweet', 'braixen', 'braviary', 'breloom', 'brionne', 'bronzong', 'bronzor', 'bruxish', 'budew', 'buizel', 'bulbasaur', 'buneary', 'bunnelby', 'burmy', 'butterfree', 'buzzwole', 'cacnea', 'cacturne', 'camerupt', 'carbink', 'carnivine', 'carracosta', 'carvanha', 

## Helper Functions

Below are a few helper function to make converting between different image data types and formats

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

## Image Classifier
A simple object detection algorithm to be applied on our images. OpenCV has a pre-trained Haar Cascade face detection model.

In [ ]:
# initialize the image classification model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
def image_classifier(filename):
    labelled_pokemon = "Pikachu"
    return labelled_pokemon

## Image Preprocessing

Prepares the image to be entered into the classification model.

In [ ]:
def image_preprocessing(img):
    # grayscale img
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # resize img
    resize_down = cv2.resize(gray, (224,224), interpolation= cv2.INTER_LINEAR)

    padded_img = cv2.copyMakeBorder(resize_down, 10, 10, 10, 10, cv2.BORDER_CONSTANT)

    return padded_img

## Capture Image from Camera

Taking image data from the webcam is straightforward.
Using Google Collab's code snippit for camera capture, we are able to take images directly from your computer's webcam.  

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))

  # get OpenCV format image
  img = js_to_image(data)

  processed_img = image_preprocessing(img)
  # save image
  cv2.imwrite(filename, processed_img)

  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()

  # Show the image which was just taken.
  display(Image(filename))
  # Classify the image and output the label
  print(image_classifier(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))
except KeyboardInterrupt:
  print("Exited due to keyboard interrupt")